In [12]:
import os

In [13]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\Detailed_Project\\Detailed_MLFLOW'

In [3]:
os.chdir('../')

In [9]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\Detailed_Project\\Detailed_MLFLOW\\research'

In [10]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: str
    unzip_data_dir: Path
    all_schema: dict

In [15]:
from src.Detailed_MLFLow_project.constants import *
from src.Detailed_MLFLow_project.utils.common import read_yaml,create_directories

class Configuration_Manager:
    def __init__(self,config_f=CONFIG_FILE_PATH,schema_f=SCHEMA_FILE_PATH,params_f=PARAMS_FILE_PATH):
        self.config_read=read_yaml(config_f)
        self.schema_read=read_yaml(schema_f)
        self.params_read=read_yaml(params_f)

        create_directories([self.config_read.artifacts_root])

    def data_validation_config(self) ->DataValidationConfig:
        config=self.config_read.data_validation
        schema=self.config_read.all_schema.keys()

        create_directories([config.root_dir])

        get_data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.status_file,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return get_data_validation_config


In [16]:
from src import logger
import pandas as pd
class Data_validation:
    def __init__(self,config:DataValidationConfig,schema_read):
        self.config=config
        self.schema_read=schema_read


    def check_data_validation(self) ->bool:
        df=pd.read_csv("artifacts/data_ingestion/data/part_names.csv")
        ref_columns=list(df.columns)
        all_schema=self.config.all_schema.keys()

        try:
            for i in ref_columns:
                if i not in all_schema():
                    validation_status=False
                    logger.info(f"feature {i} does not exist")
                    with open(self.config.status_file, "w") as stat:
                        stat.write(f"Validation Status: {validation_status}")
                
                else:
                    validation_status=True
                    logger.info(f"feature {i} matched")
                    with open(self.config.status_file, "w") as stat:
                        stat.write(f"Validation Status: {validation_status}")


        except Exception as e:
            raise e

In [19]:
try:
    con1=Configuration_Manager()
    # data_val_configure=con1.data_validation_config()
    # data_val=Data_validation(config=data_val_configure)
    # data_val.check_data_validation()
except Exception as e:
    raise e

Config file path: config.yaml


FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'